In [5]:
import tensorflow as tf
from utils import create_session_config
import tensorflow_probability as tp
import numpy as np
from models.utils import covariance,correlation_matrix,tf_cov,assert_cov_positive_definite,corr,print_tensor,print_tensors
from models.nn_pdf_common import transform_x,density_estimator,create_pdf_layer_mv,create_partially_monotone_dense_layer,create_monotone_dense_layer
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from scipy.stats import norm
from flags import FLAGS

In [18]:
tf.reset_default_graph()
global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
corr_coef = 0.7
corr_ = np.array([[1.0,corr_coef],[corr_coef,1.0]], dtype=np.float32)
std_dev = np.array([[1.0,0.0],[0,5.0]], dtype=np.float32)
true_cov = np.matmul(std_dev, np.matmul(corr_,std_dev))

data_gen = tp.distributions.MultivariateNormalFullCovariance(loc=tf.constant([0.0] * 2),
                                                                 covariance_matrix=tf.constant(true_cov))
sample_op = data_gen.sample([10])
    
data_arr = np.array([[1.0,2.0],[3.0,4.0]], dtype=np.float32)
dataset_arr = tf.data.Dataset.from_tensor_slices(data_arr)
dataset_gen = tf.data.Dataset.from_tensors(sample_op)
x_f=tf.placeholder(dtype=tf.float32, shape=[None,2])
dataset_ph = tf.data.Dataset.from_tensors(x_f)

iter = tf.data.Iterator.from_structure(dataset_arr.output_types)

y = iter.get_next()
# create the initialisation operations
arr_init_op = iter.make_initializer(dataset_arr)
gen_init_op = iter.make_initializer(dataset_gen)
ph_init_op = iter.make_initializer(dataset_ph)

with tf.Session() as sess:
    sess.run(arr_init_op) # switch to train dataset
    sess.run(y)
    sess.run(y)
    sess.run(gen_init_op) # switch to val dataset
    sess.run(y)
    sess.run(gen_init_op) 
    sess.run(y)
    sess.run(gen_init_op) 
    sess.run(y)
    sess.run(ph_init_op, feed_dict={x_f:[[7.0,8.0],[9.0,10.0]]})
    sess.run(y)

array([1., 2.], dtype=float32)

array([3., 4.], dtype=float32)

array([[-0.6760492 , -0.8348435 ],
       [-1.4923203 , -7.115209  ],
       [-0.74041617, -3.0240996 ],
       [-1.6046643 , -5.3304253 ],
       [-0.3412526 ,  1.7032733 ],
       [-0.50675654,  5.0920806 ],
       [ 0.47852707,  2.4016051 ],
       [ 0.16629736,  0.7669723 ],
       [ 1.4769595 ,  1.3136873 ],
       [ 1.2968216 ,  7.886726  ]], dtype=float32)

array([[  0.06036727,  -1.5030985 ],
       [ -0.6623494 ,  -4.810568  ],
       [  1.123797  ,   5.4944654 ],
       [ -0.09942903,   0.5205032 ],
       [  0.7757009 ,   7.214327  ],
       [ -0.1510405 ,  -4.8452973 ],
       [ -2.3672612 , -12.194161  ],
       [ -0.08104871,   0.9206013 ],
       [ -0.11757623,  -0.6340113 ],
       [  0.05100786,   6.8053346 ]], dtype=float32)

array([[-0.33647308,  0.87660855],
       [ 0.8727094 ,  5.163147  ],
       [ 0.30894616, -3.428802  ],
       [-1.7385359 , -8.656099  ],
       [ 0.8486311 ,  9.845854  ],
       [ 0.3672291 , -1.4360477 ],
       [ 0.6900572 ,  1.2616466 ],
       [-0.34113094, -1.0779765 ],
       [ 0.6137108 , 12.4429455 ],
       [ 1.2336274 , -3.130633  ]], dtype=float32)

array([[ 7.,  8.],
       [ 9., 10.]], dtype=float32)